In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    usecols=[
        'timestamp', 
        'user_id', 
        'content_id', 
        'content_type_id',
        'task_container_id'
    ],
       dtype={
           'timestamp': 'int64',
           'user_id': 'int32',
           'content_id': 'int16',
       }
)
train =train.sort_values(by=['user_id', 'timestamp'], ascending=True)
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')

In [ ]:
#userごと、時系列ごとにソートしてます
train

In [ ]:
#必要な行だけnumpy配列で取り出し
type_list = train["content_type_id"].values
content_list = train["content_id"].values
user_list = train["user_id"].values

In [ ]:
#userが変わったタイミングでフラグを立てる配列を作成
user_change_flg = []
pre_user = 0
for i in user_list:
    user = i
    if user == pre_user:
        user_change_flg.append(0)
    elif user != pre_user:
        user_change_flg.append(1)
    pre_user = user

In [ ]:
user_change_flg

In [ ]:
"""
-ざっくり流れ-
trainの上から順にcontent_type_idを確認する
lectureならlecturedには0をappendして、lectureリストにcontent_idをappend
questionならlecturedにlectureリストをappend
userがかわるごとにlectureリストを空に更新

lectured が　[[],[],[],0,[223],[223],0,[223,454],[],[],0,[451]....]　みたいになる想定
"""

lecture_list = []
lectured = []

#content_type_idを順に走査
for i, content_type in enumerate(type_list):
    #userが変わってたらlectureリスト更新
    if user_change_flg[i] == 1:
        lecture_list = []
    #questionならlecturedにlectureリストappend
    if content_type == 0:
        lectured.append(lecture_list)
    #lectureならlectureリストにcontent_idをappendして、lecturedに0をappend
    elif content_type == 1:
        lecture_list.append(content_list[i])
        lectured.append(0)
    #進捗確認用
    if i % 1000000 == 0:
        print(i)

In [ ]:
#0をはさまずにいきなりcontent_idが増えてる 逆に0の後でもcontent_id増えてないし
#原因がわからん...
lectured

In [ ]:
#lectured配列をtrainのlectured列に挿入
train["lectured"] = lectured
train

In [ ]:
"""
lecture_list = [[]*i for i in range(train["user_id_rank"].max()+1)]
#trainデータを上から順に確認して
for i in range(len(train)):
    #questionの行だった場合で
    if train["content_type_id"][i]==0:
        #回答userがlectureをuserが受けたことがあった場合は
        if lecture_list[train["user_id_rank"][i]] is not None:
            #受けたlectureのtag集をlectured列に挿入してください
            train["lectured"][i] = lecture_list[train["user_id_rank"][i]]
    #lectureの行だった場合は
    elif train["content_type_id"][i]==0:
        #意訳：受けたlectureのtagをlectureリストにappendしてください
        #直訳：該当行のcontent_idと同じ番号のlecrture_idをもつlecturesのtagをlectureリストのuser_id_rank番目にappendしてください
        lecturelist[train["user_id_rank"]].append(lectures.loc[lectures["lecture_id"]==train["content_id"][i], "tag"])
    #進捗確認用
    if i % 100==0:
        print(i)
"""

In [ ]:
ans.to_csv("./train2.csv")